**ME700: Part 1**  
Pick a problem with a known analytical solution, implement it in Fenicsx and compare the results.


Problem:
I used the same problems as hw3 part A.

A 2D rectangular beam is fixed at one end and a distributed force is applied to that beam. In small displacements, the deflection of the tip of the beam can be approximated analytically. Try to compare the analytic and numerical results.

The paprameters are selected from [the 2D linear elasticity tutorial in Fenics](https://comet-fenics.readthedocs.io/en/latest/demo/elasticity/2D_elasticity.py.html). The FenicsX code itself was taken from [this Fenicsx linear elasticity tutorial](https://bleyerj.github.io/comet-fenicsx/intro/linear_elasticity/linear_elasticity.html) and altered. 

ChatGPT was used in help with alteration.

**Importing Modules**

In [1]:
# importing modules
import numpy as np
from ufl import sym, grad, Identity, tr, inner, Measure, TestFunction, TrialFunction
from mpi4py import MPI
from dolfinx import fem, io
import dolfinx.fem.petsc
from dolfinx.mesh import create_rectangle, CellType

**Definig the Mesh and Geometry**

In [2]:
length, height = 25.0, 1.0
Nx, Ny = 250, 10
domain = create_rectangle(
    MPI.COMM_WORLD,
    [np.array([0, 0]), np.array([length, height])],
    [Nx, Ny],
    cell_type=CellType.quadrilateral,
)

**Function Space**

In [3]:
dim = domain.topology.dim
degree = 2
shape = (dim,)
V = fem.functionspace(domain, ("P", degree, shape))
u_sol = fem.Function(V, name="Displacement")

**Material + load params**

In [4]:

E_value = 1e5
nu_value = 0.3
rho = 2e-3
g = 9.81

E = fem.Constant(domain, E_value)
nu = fem.Constant(domain, nu_value)
lmbda = E * nu / (1 + nu) / (1 - 2 * nu)
mu = E / 2 / (1 + nu)


**Strain and Stress**

In [5]:
def epsilon(v): return sym(grad(v))
def sigma(v):  return lmbda * tr(epsilon(v)) * Identity(dim) + 2 * mu * epsilon(v)

**Variational Forms**

In [6]:
u = TrialFunction(V)
v = TestFunction(V)
f = fem.Constant(domain, np.array([0, -rho * g]))
dx = Measure("dx", domain=domain)
a = inner(sigma(u), epsilon(v)) * dx
L = inner(f, v) * dx

**Boundary Conditions**

In [7]:
left = lambda x: np.isclose(x[0], 0.0)
left_dofs = fem.locate_dofs_geometrical(V, left)
bcs = [fem.dirichletbc(np.zeros((2,)), left_dofs, V)]

**Solve**

In [8]:
problem = fem.petsc.LinearProblem(
    a, L, u=u_sol, bcs=bcs,
    petsc_options={"ksp_type": "preonly", "pc_type": "lu"}
)
uh = problem.solve()

**Post Process**

In [9]:

# --- Extract tip displacement at (L, h/2) ---
V1, dof_map = V.sub(1).collapse()
sub_tip = fem.locate_dofs_geometrical(
    V1, lambda x: np.isclose(x[0], length) & np.isclose(x[1], height/2)
)
global_tip = dof_map[sub_tip[0]]
tip_disp = uh.x.array[global_tip]

# --- Theoretical deflection δ = (f_vol * h) * L^4 / (8 E I) ---
f_vol = - rho * g
I = height**4 / 12
delta_th = float( (f_vol ) * length**4 / (8 * E * I) )
E_effective = E_effective = E / ( 1 - nu ** 2 )
delta_th_corrected = float( f_vol * length**4 / ( 8 * E_effective * I ) )


if MPI.COMM_WORLD.rank == 0:
    print(f"Numerical tip displacement: {tip_disp:.6e}")
    print(f"Theoretical tip displacement: {delta_th:.6e}")
    print(f"Theoretical tip displacement(corrected): {delta_th_corrected:.6e}")
    print(f"Relative error:           {abs(tip_disp - delta_th)/abs(delta_th):.2%}")
    print(f"Relative error (corrected theory):           {abs(tip_disp - delta_th_corrected)/abs(delta_th_corrected):.2%}")


Numerical tip displacement: -1.046164e-01
Theoretical tip displacement: -1.149609e-01
Theoretical tip displacement(corrected): -1.046145e-01
Relative error:           9.00%
Relative error (corrected theory):           0.00%
